In [37]:
import pickle
import pandas as pd
import os
import numpy as np

In [38]:
import pickle
import pandas as pd
import os

In [39]:
with open('ablation22', 'rb') as f:
    config_list = pickle.load(f)
non_para = 0
exist_para = 0

datasets = ['chameleon', 'cornell', 'squirrel', 'film',
            'texas', 'wisconsin', 'pubmed', 'cora', 'citeseer']
config_str = ['lr', 'do', 'es', 'wd', 'alpha', 'beta',
              'gamma', 'delta', 'nlid', 'nl', 'ordersid', 'orders']

num_map = [0, 1, 2, 3, 4, 5, 6, 9, 10, 7, 11, 8, 12, 13]

In [40]:
print('the number of all parameters', len(config_list))
results_all = [[] for _ in range(len(config_str) + 3)]
for config in config_list:
    acc_res = []
    for i in range(10):
        url_configs = ''
        for j in range(len(config_str)):
            url_configs += '_' + config_str[j] + str(config[num_map[j]])
        url = 'runs/' + datasets[config[-1]] + url_configs + \
            '_split' + str(i) + '_results.txt'
        # print(url)
        if os.path.exists(url):
            # print('url exists')
            with open(url, 'r') as f:
                s = f.read()
                sub1 = '"test_acc": '
                sub2 = ', "test_duration"'
                result = s[s.index(sub1) + len(sub1): s.index(sub2)]
                acc_res.append(float(result))
    # print(url)
    # print(len(acc_res))
    if len(acc_res) == 10:
        exist_para += 1
        results_all[0].append(datasets[config[-1]])
        for i in range(len(config_str)):
            results_all[i+1].append(config[num_map[i]])
        results_all[-2].append(np.around(np.mean(acc_res), 4))
        results_all[-1].append(np.around(np.std(acc_res), 4))
    else:
        non_para += 1
        # print(url)

print('the number of non-existing parameters', non_para)
print('the number of existing parameters', exist_para)

the number of all parameters 28
the number of non-existing parameters 17
the number of existing parameters 11


In [41]:
all_config_str = ['dataset', 'lr', 'dropout', 'early_stopping', 'weight_decay',
                  'alpha', 'beta', 'gamma', 'delta', 'norm_func_id', 'norm_layers',
                  'orders_func_id', 'orders', 'accuracy', 'std']
d = {}
assert len(all_config_str) == len(results_all)
for i in range(len(all_config_str)):
    d[all_config_str[i]] = results_all[i]

df = pd.DataFrame.from_dict(d)
df = df.sort_values(['dataset', 'accuracy']).reset_index(drop=True)

csv_name = 'results/mlpnorm_result_1+2.csv'
# if os.path.exists(csv_name):
#     df.to_csv(csv_name, mode='a', header=False)
# else:
#     df.to_csv(csv_name)

In [42]:
df

,dataset,lr,dropout,early_stopping,weight_decay,alpha,beta,gamma,delta,norm_func_id,norm_layers,orders_func_id,orders,accuracy,std
0,chameleon,0.01,0.4,300,0.00010,1.0,1.0000,0.4,1.0,2,3,2,2,0.4550,0.0161
1,chameleon,0.01,0.4,300,0.00010,1.0,0.0001,0.4,0.0,2,3,2,2,0.6901,0.0270
2,chameleon,0.01,0.4,300,0.00010,1.0,1.0000,0.4,0.0,2,3,2,2,0.7121,0.0220
3,cornell,0.01,0.0,200,0.00010,1.0,0.1000,0.7,0.0,2,2,2,2,0.4486,0.1198
4,cornell,0.01,0.0,200,0.00010,1.0,0.1000,0.7,1.0,2,2,2,2,0.8595,0.0590
5,cornell,0.01,0.0,200,0.00010,1.0,0.0001,0.7,1.0,2,2,2,2,0.8595,0.0590
6,film,0.01,0.0,40,0.00100,0.0,15000.0000,0.2,1.0,2,2,2,4,0.3770,0.0140
7,squirrel,0.01,0.8,200,0.00000,0.0,1.0000,0.0,1.0,2,3,2,2,0.3042,0.0189
8,squirrel,0.01,0.8,200,0.00000,0.0,0.0001,0.0,0.0,2,3,2,2,0.5784,0.0190
9,squirrel,0.01,0.8,200,0.00000,0.0,1.0000,0.0,0.0,2,3,2,2,0.5788,0.0176
